In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

# from Dental_Tool.Inception import create_inception_v4
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v3 import *

import keras.backend as K 
import matplotlib.pyplot as plt
import collections
import numpy as np
import keras
import time

Using TensorFlow backend.


In [2]:
directory = [ 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4_flip", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4_flip"
            ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20

def load_json(data_list, interdental=False):
            filter_fun = lambda x : { path: max(list(map(int, state))) for path, state in x.items() if max(list(map(int, state))) >= 0 }
               
            interdental_fun = lambda x : { path: state for path, state in x.items() }
            
            results = collections.OrderedDict()
            all_filtering_data, all_keys = [], []
            
            for dataset_path in data_list:
                        mapping_data = json.load(open(dataset_path , "r"))
                        filter_data  = filter_fun(mapping_data) if not interdental else interdental_fun(mapping_data)
                        all_filtering_data.append(filter_data)
                        all_keys.append( list(filter_data.keys()) )
            
            for keys in zip(*all_keys):
                    for key,  data in zip(keys, all_filtering_data):
                            results[key] = data[key]        
            return results

In [7]:
def json_2_dataframe_PBL_inderdental(data, mode=None):
        PBL_Columns = ["Path", "State", "Class", "bone_loss", "furcation", "buccal_furcation", "endo_lesion"]
        dataframe_L, dataframe_R  = pd.DataFrame(columns=PBL_Columns), pd.DataFrame(columns=PBL_Columns)
        data_dict_L, data_dict_R  = collections.OrderedDict(), collections.OrderedDict()   
        counter_L, counter_R = 0, 0
        
        molar = [1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
        premolar = [ 4, 5, 12, 13, 20, 21, 29, 30 ]
        
        less_data  = [1, 16, 17, 32]
        
        for path, info in data.items():
                state = info["state"]
                item = { 
                          "Path": path,
                          "State": state,
                          "Class": state-1 if state > 1 else 0,
                          **info
                }
                
                path_split = path.split("_")
                
#                 in_dir = path.split("/")[2]

                NN_IDX = 0
                for idx, i in enumerate(path_split):
                        if i == "NN":
                            NN_IDX = idx
                            break
                            
                original, source = "", ""
                
                if NN_IDX == 0:
                        source = "_".join(path_split[-7:-2]) 
                        original = "_".join(path_split[-7:-3])
                
                else: 
                        source = "_".join(path_split[NN_IDX:-2])
                        original = "_".join(path_split[NN_IDX:-3])
                
                item["ori_src"] = original
                item["source"] = source
#                 if 'L' == path[-5]: item["source"] = source + '_L' 
#                 elif 'R'== path[-5]: item["source"] = source + '_R'
#                 else: raise ValueError
                
                item["tooth_num"] = int(path_split[-3])
                
                if (mode == "molar"    and item["tooth_num"] not in molar   ) or (mode == "premolar" and item["tooth_num"] not in premolar): continue 
                
                item["angle"] = int(path_split[-2].split(".")[0])
                
                if 'L' == path[-5]:
                        data_dict_L[counter_L] = item
                        counter_L += 1        
                
                if 'R' == path[-5]:
                        data_dict_R[counter_R] = item
                        counter_R += 1        

        
        dataframe_L = dataframe_L.from_dict(data_dict_L, "index")
        dataframe_R = dataframe_R.from_dict(data_dict_R, "index")
        
        dataframe = pd.concat([dataframe_L, dataframe_R])
        
        return dataframe

In [8]:
data = load_json(directory, interdental=True)
dataset = json_2_dataframe_PBL_inderdental(data)
dataset = dataset[ (dataset["State"] >=0) & (dataset["state"] >=0) & (dataset["bone_loss"] >=0) ].reset_index(drop=True)
# dataset.to_csv("tmp.csv", index=False)
dataset 

,Path,State,Class,state,bone_loss,furcation,buccal_furcation,endo_lesion,ori_src,source,tooth_num,angle
0,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,-10
1,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,-10
2,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,-10
3,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,-10
4,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,-9
...,...,...,...,...,...,...,...,...,...,...,...,...
629995,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,8
629996,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,9
629997,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,9
629998,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,9


In [10]:
# # tooth_group = dataset.groupby("source")
# groups = [ tooth_table for ori_src, table in dataset.groupby("ori_src") for src, tooth_table in table.groupby("source") ]
# # random.shuffle(groups)
# dataset = pd.concat(groups).reset_index(drop=True)
# dataset.to_csv("tmp.csv", index=True)
    
        
        

In [26]:
# import glob
# import os

# path = "Dental_Data/PBL/Ori_Seg"
# change = "Ori"
# Ori = [ i.replace("\\", "/") for i in glob.iglob(f"{path}/{change}/*") ]
# Ori
# # Rename = [ f"{path}/{change}/" + "_".join(i.split("/")[4].split("_")[1:] ) for i in Ori ]

# # for src, dst in zip(Ori, Rename): 
# #     os.rename(src, dst)

['Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151623_BE78A8_6_5.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151623_BE78A8_7_5.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151627_BE78AD_25_-2.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151627_BE78AD_26_0.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151631_BE78BA_2_-24.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151636_BE78B6_2_-11.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151636_BE78B6_3_-19.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151636_BE78B6_6_-5.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151652_BE78B0_6_0.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151656_BE78AE_30_35.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151657_BE78BB_11_27.png',
 'Dental_Data/PBL/Ori_Seg/Ori/000408 102419 x_NN_191024_151657_BE78BB_12_29.png',
 'Dental_Data/PBL/Ori_Se

In [8]:
# directory = [ 
#                 "Dental_Data/PBL/10_20201015_multilabel", 
#                 "Dental_Data/PBL/10_20201015_multilabel_flip", 
#                 "Dental_Data/PBL/10_20201015_clahe_multilabel", 
#                 "Dental_Data/PBL/10_20201015_clahe_multilabel_flip"
#             ]

directory = [ 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4_flip", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4_flip"
            ]
directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20

def load_json(data_list, interdental=False):
            filter_fun = lambda x : { path: max(list(map(int, state))) for path, state in x.items() if max(list(map(int, state))) >= 0 }
               
            interdental_fun = lambda x : { path: state for path, state in x.items() }
            
            results = collections.OrderedDict()
            all_filtering_data, all_keys = [], []
            
            for dataset_path in data_list:
                        mapping_data = json.load(open(dataset_path , "r"))
                        filter_data  = filter_fun(mapping_data) if not interdental else interdental_fun(mapping_data)
                        all_filtering_data.append(filter_data)
                        all_keys.append( list(filter_data.keys()) )
            
            for keys in zip(*all_keys):
                    for key,  data in zip(keys, all_filtering_data):
                            results[key] = data[key]        
            return results
        
def json_2_dataframe_PBL_Multitask(data, mode=None):
        PBL_Columns = ["Path", "State", "Class", "bone_loss", "furcation", "buccal_furcation", "endo_lesion"]
        dataframe = pd.DataFrame(columns=PBL_Columns)
        data_dict = collections.OrderedDict()
        counter = 0
        
        molar = [1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
        premolar = [ 4, 5, 12, 13, 20, 21, 29, 30 ]
        
        less_data  = [1, 16, 17, 32]
        
        for path, info in data.items():
                state = max(info["state"])
                bone_loss = max(info["bone_loss"])
                item = { 
                          "Path": path,
                          "State": state,
                          "Class": state-1 if state > 1 else 0,
                          "bone_loss": bone_loss 
                }
                
                path_split = path.split("_")
                
#                 in_dir = path.split("/")[2]

                NN_IDX = 0
                for idx, i in enumerate(path_split):
                        if i == "NN":
                            NN_IDX = idx
                            break
                
                original, source = "", ""
                
                if NN_IDX == 0:
                        source = "_".join(path_split[-6:-1]) 
                        original = "_".join(path_split[-6:-2])
                
                else: 
                        source = "_".join(path_split[NN_IDX:-1])
                        original = "_".join(path_split[NN_IDX:-2])
                
                item["ori_src"] = original
                item["source"] = source
                item["tooth_num"] = int(path_split[-2])
                
                if (mode == "molar" and (item["tooth_num"] not in molar)   ) or (mode == "premolar" and (item["tooth_num"] not in premolar) ): continue 
                    
                item["angle"] = int(path_split[-1].split(".")[0])
                
                data_dict[counter] = item
                counter += 1        

        dataframe = dataframe.from_dict(data_dict, "index")
        
        return dataframe
    
def json_2_dataframe_PBL_inderdental(data, mode=None):
        PBL_Columns = ["Path", "State", "Class", "bone_loss", "furcation", "buccal_furcation", "endo_lesion"]
        dataframe_L, dataframe_R  = pd.DataFrame(columns=PBL_Columns), pd.DataFrame(columns=PBL_Columns)
        data_dict_L, data_dict_R  = collections.OrderedDict(), collections.OrderedDict()   
        counter_L, counter_R = 0, 0
        
        molar = [1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
        premolar = [ 4, 5, 12, 13, 20, 21, 29, 30 ]
        
        less_data  = [1, 16, 17, 32]
        
        for path, info in data.items():
                state = info["state"]
                item = { 
                          "Path": path,
                          "State": state,
                          "Class": state-1 if state > 1 else 0,
                          **info
                }
                
                path_split = path.split("_")
                
#                 in_dir = path.split("/")[2]

                NN_IDX = 0
                for idx, i in enumerate(path_split):
                        if i == "NN":
                            NN_IDX = idx
                            break
                
                original, source = "", ""
                
                if NN_IDX == 0:
                        source = "_".join(path_split[-7:-2]) 
                        original = "_".join(path_split[-7:-3])
                
                else: 
                        source = "_".join(path_split[NN_IDX:-2])
                        original = "_".join(path_split[NN_IDX:-3])
                
                item["ori_src"] = original
                item["source"] = source
                item["side"] = source + "_" + path[-5]
#                 if 'L' == path[-5]: item["source"] = source + '_L' 
#                 elif 'R'== path[-5]: item["source"] = source + '_R'
#                 if 
#                 else: raise ValueError
                
                item["tooth_num"] = int(path_split[-3])
                
                if (mode == "molar"    and item["tooth_num"] not in molar   ) or (mode == "premolar" and item["tooth_num"] not in premolar): continue 
                
                item["angle"] = int(path_split[-2].split(".")[0])
                
                if 'L' == path[-5]:
                        data_dict_L[counter_L] = item
                        counter_L += 1        
                
                if 'R' == path[-5]:
                        data_dict_R[counter_R] = item
                        counter_R += 1        

        
        dataframe_L = dataframe_L.from_dict(data_dict_L, "index")
        dataframe_R = dataframe_R.from_dict(data_dict_R, "index")
        
        dataframe = pd.concat([dataframe_L, dataframe_R])
        
        return dataframe
    

In [9]:
data = load_json(directory, interdental=True)
dataset = json_2_dataframe_PBL_inderdental(data)
# dataset = json_2_dataframe_PBL_Multitask(data)
dataset = dataset[ (dataset["State"] >=0) & (dataset["State"] >=0) & (dataset["bone_loss"] >=0) ]
dataset

,Path,State,Class,state,bone_loss,furcation,buccal_furcation,endo_lesion,ori_src,source,side,tooth_num,angle
0,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,NN_191024_151623_BE78A8_6_L,6,-10
1,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,NN_191024_151623_BE78A8_6_L,6,-10
2,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,NN_191024_151623_BE78A8_6_L,6,-10
3,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,NN_191024_151623_BE78A8_6_L,6,-10
4,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,-99,-99,-99,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,NN_191024_151623_BE78A8_6_L,6,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
388075,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,NN_180917_113933_C0A0B2_26_R,26,8
388076,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,NN_180917_113933_C0A0B2_26_R,26,9
388077,Dental_Data/PBL/10_interdental_20201015_multil...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,NN_180917_113933_C0A0B2_26_R,26,9
388078,Dental_Data/PBL/10_interdental_20201015_clahe_...,1,0,1,1,0,0,0,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,NN_180917_113933_C0A0B2_26_R,26,9


In [20]:
cmx = np.zeros((3, 3))
        
for fur in range(2):
        cmx = np.zeros((3, 3))
        for stage in range(3):
                for bone_loss in range(3):
                         cmx_item = dataset[ (dataset["Class"]==stage) & (dataset["bone_loss"]==bone_loss) & (dataset["furcation"]==fur) ]
                         cmx[stage, bone_loss] = len(cmx_item)
        print(cmx)
        

[[ 67760. 141760.  30720.]
 [     0.  32000.  14400.]
 [     0.   8880.   7600.]]
[[ 320. 6080. 1760.]
 [  80. 2000. 2480.]
 [   0. 1840. 2480.]]


In [25]:
for i in range(1, 6, 1):
        path = f"Results/Inception_Multitask_Fold_520201021-115143/Fold_{i}/table/test_predict.csv"
        data = pd.read_csv(path)
        
        print(f"==================== Fold_{i} ========================")
        matches = []
        for gt, predict in zip(data.Class, data.Predict):
                tmp = 1 if gt == predict else 0
                matches.append(tmp)
        
        data["Match"] = matches
        data_filter = data[ data["furcation"] >=0 ]
        
        
        for fur in range(2):
                print(f"Furcation {fur}")
                cmx = np.zeros((3, 3))
                for stage in range(3):
                        for bone_loss in range(3):
                                 total = data_filter[ (data_filter["Class"]==stage) & (data_filter["bone_loss"]==bone_loss) & (data_filter["furcation"]==fur) ]
                                 cmx_item = data_filter[ (data_filter["Class"]==stage) & (data_filter["bone_loss"]==bone_loss) & (data_filter["furcation"]==fur) & (data_filter["Match"] == 1) ]        
                                 if len( total ) != 0 :  cmx[stage, bone_loss] = round(len(cmx_item) / len(total) * 100, 2)
                print(cmx)
                
                
                
        for fur in range(2):
                print(f"Furcation {fur} -------- total")
                cmx = np.zeros((3, 3))
                for stage in range(3):
                        for bone_loss in range(3):
                                 total = data_filter[ (data_filter["Class"]==stage) & (data_filter["bone_loss"]==bone_loss) & (data_filter["furcation"]==fur) ]
                                 cmx[stage, bone_loss] = len(total)
                print(cmx)
                
        
        for fur in range(2):
                print(f"Furcation {fur} --------  match")
                cmx = np.zeros((3, 3))
                for stage in range(3):
                        for bone_loss in range(3):
                                 total = data_filter[ (data_filter["Class"]==stage) & (data_filter["bone_loss"]==bone_loss) & (data_filter["furcation"]==fur)  & (data_filter["Match"] == 1) ]
                                 cmx[stage, bone_loss] = len(total)
                print(cmx)
                
                
        
        for fur in range(2):
                print(f"Furcation {fur}")
                total = data_filter[ (data_filter["furcation"]==fur) ]
                cmx_item = data_filter[ (data_filter["furcation"]==fur) & (data_filter["Match"] == 1) ]
                print(len(total), len(cmx_item))
                if len( total ) != 0 : print(round(len(cmx_item) / len(total) * 100, 2))

                    

==================== Fold_1 ========================
Furcation 0
[[97.23 84.86 87.5 ]
 [ 0.   54.77 69.21]
 [ 0.   70.3  76.01]]
Furcation 1
[[ 0.   53.7  62.5 ]
 [ 0.   55.07 38.08]
 [ 0.   96.25 88.75]]
Furcation 0 -------- total
[[2199. 1724.  384.]
 [   0. 2850. 1166.]
 [   0. 1680. 1680.]]
Furcation 1 -------- total
[[  0.  54.  24.]
 [  0. 207. 239.]
 [  0. 240. 720.]]
Furcation 0 --------  match
[[2138. 1463.  336.]
 [   0. 1561.  807.]
 [   0. 1181. 1277.]]
Furcation 1 --------  match
[[  0.  29.  15.]
 [  0. 114.  91.]
 [  0. 231. 639.]]
Furcation 0
11683 8763
75.01
Furcation 1
1484 1119
75.4
==================== Fold_2 ========================
Furcation 0
[[98.2  77.97 78.49]
 [ 0.   72.47 79.49]
 [ 0.   49.73 56.02]]
Furcation 1
[[ 0.   60.   47.06]
 [ 0.   76.75 55.07]
 [ 0.   41.   70.  ]]
Furcation 0 -------- total
[[1946. 1643.  358.]
 [   0. 2677. 1302.]
 [   0. 1840. 1280.]]
Furcation 1 -------- total
[[  0.  95.  34.]
 [  0. 314. 207.]
 [  0. 800.  80.]]
Furcation 0 -

In [34]:
from Dental_Tool.Dataloader import json_2_dataframe_PBL, load_json, init_directory
directory = [ "Dental_Data/PBL/furcation" ]
directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20
data = load_json(directory, interdental=False)
dataset = json_2_dataframe_PBL(data)
fur = dataset[ dataset["State"] == 1 ].reset_index(drop=True)
no_fur = dataset[ dataset["State"] == 0 ].reset_index(drop=True)

In [35]:
fur

,Path,State,Class,source,tooth_num,angle
0,Dental_Data/PBL/furcation/06-34-31-182_000408 ...,1,0,NN_191024_151631_BE78BA_2,2,-24
1,Dental_Data/PBL/furcation/06-34-31-226_000408 ...,1,0,NN_191024_151636_BE78B6_2,2,-11
2,Dental_Data/PBL/furcation/06-34-32-664_001742 ...,1,0,NN_120827_095324_BFFF2E_3,3,-22
3,Dental_Data/PBL/furcation/06-34-32-696_001742 ...,1,0,NN_120827_095324_BFFF2E_5,5,-27
4,Dental_Data/PBL/furcation/06-34-32-791_001742 ...,1,0,NN_120827_095333_BFFF2F_2,2,-9
...,...,...,...,...,...,...
354,Dental_Data/PBL/furcation/06-36-07-490_S111831...,1,0,NN_190410_101528_AC0ED1_31,31,10
355,Dental_Data/PBL/furcation/06-36-08-413_S112552...,1,0,NN_121025_095409_AC3B0F_26,26,-3
356,Dental_Data/PBL/furcation/06-36-08-530_S112552...,1,0,NN_121025_095501_AC3B15_26,26,-7
357,Dental_Data/PBL/furcation/06-36-09-287_S117029...,1,0,NN_140609_112257_C0A542_13,13,7


In [39]:
# target_dir = "Dental_Data/PBL/furcation/no_fur"
# init_directory(target_dir)
# for path in no_fur["Path"]:
#         image_name = os.path.basename(path)
#         image = cv2.imread(path, 0)
#         cv2.imwrite(f"{target_dir}/{image_name}", image)